Pseudocode for this section is written below.
Here are some useful links
    - https://www.instructables.com/Plotting-real-time-data-from-Arduino-using-Python-/ // shows an example of live matplotlib plotting through serial
        - https://gist.github.com/electronut/d5e5f68c610821e311b0
    - https://stackoverflow.com/questions/66355982/plot-real-time-serial-data-using-python-with-arduino // we love and appreciate StackOverflow
    - https://stackoverflow.com/questions/24838311/plotting-serial-data-using-arduino-and-pyserial // we love and appreciate StackOverflow


// decode this data further so that it can be used with the matplotlib code
// make it easier to stop and start the flow of data through combinations of key presses (for example)
// understand how to send transformed data across back from the serial  
   // in a different way/different channel since I want this to pass through an (e.g.) ESP to send data to haptic motors
   // need to understand how to communicate between multiple pieces of code automatically
   // to display this clearly


-----------
// clean data
    // functions needed to make life easy
    - strip() # strip the tabs on the between the data points
    - //be aware of the "|" in the middle 
    - isdigit()

    // sort out tilt data and accelerometer
        // how can this be interpreted
        // how do I want this displayed
            - a plane of unit length 1 
            or
            - using it as a vector so that eventually it can be approximated as a curve
                - look into spline curves and how it can be approximated using cubic splines
                https://www.asprs.org/wp-content/uploads/2010/12/Blonquist.pdf
                https://math.stackexchange.com/questions/577641/how-to-calculate-interpolating-splines-in-3d-space
                    // the above is a better one

    //

// plot in matplotlib with time delay
    // what am I doing with the tilt data and accelerometer data




Below will be code that reads in data from the serial (from the Arduino Uno) to be read and manipulated in Thonny's terminal

The code in the ArduinoIDE reading in the position of the accelerometer and gyroscope. 
N.B. This was written using ClaudeAI since I am not proficient in C++ and needed a fast way to test the code (before I dedicate time to learning it).


REMEBER TO WRITE UP THE PROMPT THAT I USED FOR THIS (SPECIFICALLY (or just rewrite the prompt if it was part of segmented aspects))

In [ ]:
#include <Wire.h>

// TCA9548A I2C Multiplexer address
#define TCA9548A_ADDR 0x70
#define MPU6050_ADDR 0x68

// MPU6050 registers
#define PWR_MGMT_1 0x6B
#define ACCEL_XOUT_H 0x3B
#define WHO_AM_I 0x75

// Maximum channels to check
#define MAX_CHANNELS 8

// Active sensor tracking
uint8_t activeSensors = 0;
uint8_t sensorCount = 0;

void setup() {
  Serial.begin(9600);
  Wire.begin();
  
  Serial.println(F("=== Enhanced TCA9548A + MPU6050 Diagnostic ==="));
  delay(100);
  
  // Test TCA9548A connection
  Serial.print(F("Testing TCA9548A at 0x"));
  Serial.print(TCA9548A_ADDR, HEX);
  Serial.print(F("... "));
  
  Wire.beginTransmission(TCA9548A_ADDR);
  if (Wire.endTransmission() != 0) {
    Serial.println(F("FAILED!"));
    Serial.println(F("Check: VCC, GND, SDA->A4, SCL->A5, A0/A1/A2->GND"));
    while(1);
  }
  Serial.println(F("OK"));
  
  // Comprehensive channel scan
  Serial.println(F("\n=== Detailed Channel Scan ==="));
  sensorCount = 0;
  activeSensors = 0;
  
  for (uint8_t ch = 0; ch < MAX_CHANNELS; ch++) {
    Serial.print(F("Channel "));
    Serial.print(ch);
    Serial.print(F(": "));
    
    // Select channel
    selectChannel(ch);
    delay(50); // Longer delay for stability
    
    // Scan for any I2C device
    bool deviceFound = false;
    for (uint8_t addr = 0x08; addr < 0x78; addr++) {
      Wire.beginTransmission(addr);
      if (Wire.endTransmission() == 0) {
        if (!deviceFound) {
          Serial.print(F("Device(s) at: "));
          deviceFound = true;
        }
        Serial.print(F("0x"));
        Serial.print(addr, HEX);
        Serial.print(F(" "));
      }
    }
    
    if (!deviceFound) {
      Serial.println(F("No devices"));
      continue;
    }
    
    // Check specifically for MPU6050
    Wire.beginTransmission(MPU6050_ADDR);
    uint8_t error = Wire.endTransmission();
    
    if (error == 0) {
      // Try to read WHO_AM_I
      Wire.beginTransmission(MPU6050_ADDR);
      Wire.write(WHO_AM_I);
      Wire.endTransmission(false);
      
      uint8_t bytesReceived = Wire.requestFrom(MPU6050_ADDR, 1, true);
      
      if (bytesReceived == 1) {
        uint8_t id = Wire.read();
        Serial.print(F("| MPU6050 ID: 0x"));
        Serial.print(id, HEX);
        
        if (id == 0x68 || id == 0x72) {
          Serial.print(F(" ✓"));
          activeSensors |= (1 << ch);
          sensorCount++;
          
          // Initialize this sensor
          if (initMPU6050()) {
            Serial.print(F(" INIT_OK"));
          } else {
            Serial.print(F(" INIT_FAIL"));
          }
        } else {
          Serial.print(F(" WRONG_ID"));
        }
      } else {
        Serial.print(F("| MPU6050: NO_RESPONSE"));
      }
    } else {
      Serial.print(F("| MPU6050: ERROR_"));
      Serial.print(error);
    }
    
    Serial.println();
  }
  
  Serial.print(F("\n=== Summary: "));
  Serial.print(sensorCount);
  Serial.println(F(" working MPU6050 sensors found ==="));
  
  if (sensorCount == 0) {
    Serial.println(F("\nTROUBLESHOOTING:"));
    Serial.println(F("1. Check power connections (3.3V/5V)"));
    Serial.println(F("2. Verify ground connections"));
    Serial.println(F("3. Check SDA/SCL wiring to each channel"));
    Serial.println(F("4. Try different channels"));
    Serial.println(F("5. Test each MPU6050 individually"));
    while(1);
  }
  
  Serial.println(F("\nStarting continuous readings..."));
  delay(2000);
}

void loop() {
  static unsigned long lastRead = 0;
  
  if (millis() - lastRead >= 1000) {
    lastRead = millis();
    
    Serial.print(F("Time: "));
    Serial.print(millis()/1000);
    Serial.println(F("s"));
    
    // Read each active sensor
    for (uint8_t ch = 0; ch < MAX_CHANNELS; ch++) {
      if (activeSensors & (1 << ch)) {
        readAndDisplaySensor(ch);
      }
    }
    Serial.println();
  }
}

void selectChannel(uint8_t channel) {
  if (channel >= 8) return;
  
  Wire.beginTransmission(TCA9548A_ADDR);
  Wire.write(1 << channel);
  uint8_t error = Wire.endTransmission();
  
  if (error != 0) {
    Serial.print(F("Channel select error: "));
    Serial.println(error);
  }
}

bool initMPU6050() {
  // Wake up MPU6050
  Wire.beginTransmission(MPU6050_ADDR);
  Wire.write(PWR_MGMT_1);
  Wire.write(0x00);
  if (Wire.endTransmission() != 0) return false;
  
  delay(10);
  
  // Set accelerometer range to ±2g
  Wire.beginTransmission(MPU6050_ADDR);
  Wire.write(0x1C);
  Wire.write(0x00);
  if (Wire.endTransmission() != 0) return false;
  
  // Set gyroscope range to ±250°/s
  Wire.beginTransmission(MPU6050_ADDR);
  Wire.write(0x1B);
  Wire.write(0x00);
  if (Wire.endTransmission() != 0) return false;
  
  return true;
}

void readAndDisplaySensor(uint8_t channel) {
  selectChannel(channel);
  delay(10);
  
  // Read sensor data
  Wire.beginTransmission(MPU6050_ADDR);
  Wire.write(ACCEL_XOUT_H);
  if (Wire.endTransmission(false) != 0) {
    Serial.print(F("S"));
    Serial.print(channel);
    Serial.println(F(": READ_ERROR"));
    return;
  }
  
  uint8_t bytesReceived = Wire.requestFrom(MPU6050_ADDR, 14, true);
  
  if (bytesReceived != 14) {
    Serial.print(F("S"));
    Serial.print(channel);
    Serial.print(F(": BYTES_ERROR (got "));
    Serial.print(bytesReceived);
    Serial.println(F(")"));
    return;
  }
  
  // Read all 14 bytes (6 accel + 2 temp + 6 gyro)
  int16_t ax = (Wire.read() << 8) | Wire.read();
  int16_t ay = (Wire.read() << 8) | Wire.read();
  int16_t az = (Wire.read() << 8) | Wire.read();
  int16_t temp = (Wire.read() << 8) | Wire.read();
  int16_t gx = (Wire.read() << 8) | Wire.read();
  int16_t gy = (Wire.read() << 8) | Wire.read();
  int16_t gz = (Wire.read() << 8) | Wire.read();
  
  // Display results
  Serial.print(F("S"));
  Serial.print(channel);
  Serial.print(F(": A="));
  Serial.print(ax/16384.0, 2);
  Serial.print(F(","));
  Serial.print(ay/16384.0, 2);
  Serial.print(F(","));
  Serial.print(az/16384.0, 2);
  
  Serial.print(F(" G="));
  Serial.print(gx/131.0, 1);
  Serial.print(F(","));
  Serial.print(gy/131.0, 1);
  Serial.print(F(","));
  Serial.print(gz/131.0, 1);
  
  Serial.print(F(" T="));
  Serial.print((temp/340.0) + 36.53, 1);
  Serial.println(F("C"));
}

This is the code in Python to read this data -- based on the context of this tutorial: 
https://www.youtube.com/watch?v=AHr94RtMj1A&t=39s

In [ ]:
import serial.tools.list_ports

ports = serial.tools.list_ports.comports()
serialInst = serial.Serial()

portList = []

for port in ports:
    portList.append(str(port))
    print(str(port))
    
val = input("Select port: COM")

for x in range(len(portList)):
    if portList[x].startswith("COM" + str(val)):
        portVar = "COM" + str(val)
        print(portList[x])
        
serialInst.baudrate = 9600
serialInst.port = portVar
serialInst.open()

while True:
    if serialInst.in_waiting: # if there is data in the buffer
        packet = serialInst.readline()
        decoded = packet.decode('utf')
        print(decoded)
        
# decode this data further so that it can be used with the matplotlib code
# make it easier to stop and start the flow of data through combinations of key presses (for example)
# understand how to send transformed data across back from the serial  
   # in a different way/different channel since I want this to pass through an (e.g.) ESP to send data to haptic motors
   # need to understand how to communicate between multiple pieces of code automatically
   # to display this clearly


